In [4]:

from google.colab import files
import os

CSV_PATH = "/content/load_balancing_realistic_sla.csv"
if not os.path.exists(CSV_PATH):
    print("Upload load_balancing_realistic_sla.csv")
    uploaded = files.upload()
    os.rename(list(uploaded.keys())[0], CSV_PATH)

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.read_csv(CSV_PATH).reset_index(drop=True)

NUM_CPUS = 4
WINDOW = 5
DRIFT_THRESHOLD = 0.30


for i in range(NUM_CPUS):
    df[f"cpu_{i}_load_sma"] = df[f"cpu_{i}_load"].rolling(WINDOW).mean()
    df[f"cpu_{i}_load_ewma"] = df[f"cpu_{i}_load"].ewm(span=WINDOW).mean()

df.fillna(method="bfill", inplace=True)



models = {}
for i in range(NUM_CPUS):
    X = df[[f"cpu_{i}_load_sma", f"cpu_{i}_load_ewma", "system_load_avg"]]
    y = df[f"cpu_{i}_load"]
    model = LinearRegression()
    model.fit(X, y)
    models[i] = model



def round_robin(t):
    return t % NUM_CPUS

def lwr(row):
    eff = [
        row[f"cpu_{i}_load"] / row[f"cpu_{i}_speed"]
        for i in range(NUM_CPUS)
    ]
    return int(np.argmin(eff))

def drift_aware(row):
    drift = False
    predicted = []

    for i in range(NUM_CPUS):
        Xp = pd.DataFrame([{
            f"cpu_{i}_load_sma": row[f"cpu_{i}_load_sma"],
            f"cpu_{i}_load_ewma": row[f"cpu_{i}_load_ewma"],
            "system_load_avg": row["system_load_avg"]
        }])

        pred = models[i].predict(Xp)[0]
        predicted.append(pred)

        if abs(pred - row[f"cpu_{i}_load"]) / (row[f"cpu_{i}_load"] + 1e-6) > DRIFT_THRESHOLD:
            drift = True

    if drift:
        eff = [predicted[i] / row[f"cpu_{i}_speed"] for i in range(NUM_CPUS)]
        return int(np.argmin(eff))
    else:
        return lwr(row)

def evaluate(policy):
    total_reward = 0
    sla_violations = 0
    oracle_hits = 0

    for t, row in df.iterrows():
        if policy == "RR":
            cpu = round_robin(t)
        elif policy == "LWR":
            cpu = lwr(row)
        else:
            cpu = drift_aware(row)

        completion = (
            row[f"cpu_{cpu}_wait_time"]
            + row["task_size"] / row[f"cpu_{cpu}_speed"]
        )

        sla = int(completion > row["sla_deadline"])

        total_reward += -completion - 5.0 * sla
        sla_violations += sla
        oracle_hits += int(cpu == row["chosen_cpu_oracle"])

    n = len(df)
    return (
        total_reward / n,
        oracle_hits / n * 100,
        sla_violations / n * 100
    )



rr = evaluate("RR")
lwr_res = evaluate("LWR")
drift = evaluate("DRIFT")

print("\n=== ROUND ROBIN ===")
print(f"Avg Reward      : {rr[0]:.2f}")
print(f"Oracle Accuracy : {rr[1]:.2f}%")
print(f"SLA Violation % : {rr[2]:.2f}%")

print("\n=== LWR ===")
print(f"Avg Reward      : {lwr_res[0]:.2f}")
print(f"Oracle Accuracy : {lwr_res[1]:.2f}%")
print(f"SLA Violation % : {lwr_res[2]:.2f}%")

print("\n=== DRIFT-AWARE PREDICTIVE ===")
print(f"Avg Reward      : {drift[0]:.2f}")
print(f"Oracle Accuracy : {drift[1]:.2f}%")
print(f"SLA Violation % : {drift[2]:.2f}%")


/tmp/ipython-input-349877894.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)



=== ROUND ROBIN ===
Avg Reward      : -9.05
Oracle Accuracy : 25.78%
SLA Violation % : 63.66%

=== LWR ===
Avg Reward      : -6.30
Oracle Accuracy : 71.16%
SLA Violation % : 42.62%

=== DRIFT-AWARE PREDICTIVE ===
Avg Reward      : -6.77
Oracle Accuracy : 58.36%
SLA Violation % : 46.73%
